# Иерархия памяти CPU

Попытаемся ответить на вопрос, что происходит в железе, когда мы выполняем такой код, сколько тактов он работает:

```c++
int b = 5;

...;

int a = b + 3;  // b - переменная на стеке (предположим, компилятор во время оптимизаций её не выкинул)
```

_(Рисовать как дотянуть переменную `b` из RAM до регистров и зачем нужны кеши)_

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />


![](mem_hierarchy_single_cpu.png)

<br />

**Cache line**:

Когда данные идут из памяти в регистры через кеши, они идут блоками по (обычно) 64 байта. Этими же блоками они оседают в кешах памяти. Такой блок называется cache line.

`64 byte == 8 x int64_t == 8 x double == 16 x int32_t == 16 x float`

Рассмотрим функцию, суммирующую элементы массива:

```c++
float sum(const std::vector<float>& v)
{
    float rv = 0;
    for (float x : v)
        rv += x;
    return rv;
}
```

_(Рисовать как будут бегать данные между памятью и кешами для этой функции)_

<br />

## Многопоточность и cache line-ы. Разреженность

Рассмотрим две очень похожие функции, суммирующие вектор в 4 потока:

```c++
unsigned sum_4T_cache_hits(const std::vector<unsigned>& v) {
  const unsigned n = v.size();

  std::vector<unsigned> sums(4, 0);

  const auto calc_sum = [&v, &sums](
      unsigned start_ix,
      unsigned final_ix,
      unsigned sum_ix) {
    unsigned &psum = sums[sum_ix];
    for (unsigned i = start_ix; i < final_ix; ++i)
      psum += v[i];
  };

  std::thread t1(calc_sum, n * 0/4, n * 1/4, 0);
  std::thread t2(calc_sum, n * 1/4, n * 2/4, 1);
  std::thread t3(calc_sum, n * 2/4, n * 3/4, 2);
  std::thread t4(calc_sum, n * 3/4, n * 4/4, 3);
  t1.join(); t2.join(); t3.join(); t4.join();

  return sums[0] + sums[1] + sums[2] + sums[3];
}
```

```c++
unsigned sum_4T_no_cache_hits(const std::vector<unsigned>& v) {
  const unsigned n = v.size();

  std::vector<unsigned> sums(1024, 0);

  const auto calc_sum = [&v, &sums](
      unsigned start_ix,
      unsigned final_ix,
      unsigned sum_ix) {
    unsigned s = 0;
    for (unsigned i = start_ix; i < final_ix; ++i)
      s += v[i];
    sums[sum_ix] = s;
  };

  std::thread t1(calc_sum, n * 0/4, n * 1/4, 0);
  std::thread t2(calc_sum, n * 1/4, n * 2/4, 203);
  std::thread t3(calc_sum, n * 2/4, n * 3/4, 405);
  std::thread t4(calc_sum, n * 3/4, n * 4/4, 978);
  t1.join(); t2.join(); t3.join(); t4.join();

  return sums[0] + sums[203] + sums[405] + sums[978];
}
```

Вторая реализация работает (на моей локальной машине) на 14% быстрее.

_объяснить причину, нарисовать_

<br />

**За пределами рассказа**:
* ассоциативность кешей
* NUMA